In [1]:
# Imports:
import sys
import warnings
from os import getcwd

import datetime as dt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

# Skyfield
from skyfield.api import EarthSatellite, load
from sgp4.api import Satrec, WGS72
from sgp4.model import wgs72, wgs84
from sgp4.conveniences import jday_datetime, UTC, sat_epoch_datetime, dump_satrec
from sgp4 import exporter

# Astropy
from astropy.coordinates import SkyCoord, TEME, TETE
from astropy.time import Time

import astropy
print(astropy.__file__)

# Load the GMAT Python API
sys.path.append('C:/Work/Programme/GMAT_R2022a/GMAT/api/')
from load_gmat import *

# Load helper functions
from src.functions import *

# TEME -> True Equator Mean Equinox
# TETE -> True of date / True Equator True Equinox
# https://docs.astropy.org/en/stable/coordinates/index.html#module-astropy.coordinates.builtin_frames
# conversion from TETE (TOD) to TEME

c:\Users\franz\repos\spaceops-orbit-determination\.venv\lib\site-packages\astropy\__init__.py


In [2]:
# Read ephemeris from GPS
gps_data = pd.read_excel('02_Data/ephemeris/ephemeris_gps.xlsx')
display(gps_data)

,Unnamed: 0,Datetime_UTC,pos_x,pos_y,pos_z,vel_x,vel_y,vel_z
0,0,2024-06-06 17:50:01.200,-1734.48514,1054.75763,6534.02166,-3786.88536,6311.83227,-2021.78652
1,1,2024-06-06 17:50:31.200,-1847.10493,1243.49085,6469.72896,-3720.35961,6269.10939,-2264.07250
2,2,2024-06-06 17:51:01.200,-1957.66656,1430.83867,6398.20526,-3649.65933,6219.49385,-2503.64783
3,3,2024-06-06 17:51:31.200,-2066.03743,1616.57661,6319.53764,-3574.89072,6162.82842,-2740.60896
4,4,2024-06-06 17:52:01.200,-2172.12254,1800.54230,6233.79323,-3496.17482,6099.31807,-2974.64289
...,...,...,...,...,...,...,...,...
2618,2618,2024-06-07 16:59:02.200,3530.21695,-5627.71898,1696.90092,-1888.17292,1040.27195,7315.21742
2619,2619,2024-06-07 16:59:32.200,3471.61703,-5593.38228,1915.36965,-2018.05189,1248.41287,7247.89441
2620,2620,2024-06-07 17:00:02.200,3409.14964,-5552.81798,2131.70063,-2145.99050,1455.58871,7172.54534
2621,2621,2024-06-07 17:00:32.200,3342.88568,-5506.06835,2345.64762,-2271.59521,1661.35697,7089.16409


In [3]:
# Position transformation

# Convert 'Datetime_UTC' to astropy Time
datetime_utc = Time(pd.to_datetime(gps_data["Datetime_UTC"]))

# Position transformation
pos = SkyCoord(
    x=gps_data["pos_x"].values,
    y=gps_data["pos_y"].values,
    z=gps_data["pos_z"].values,
    representation_type='cartesian',
    frame='tete',
    obstime=datetime_utc,
    unit='km'
)

# Transform from TETE to TEME frame
pos_teme = pos.transform_to(TEME(obstime=datetime_utc))

# Velocity transformation
vel = SkyCoord(
    x=gps_data["vel_x"].values,
    y=gps_data["vel_y"].values,
    z=gps_data["vel_z"].values,
    representation_type='cartesian',
    frame='tete',
    obstime=datetime_utc,
    unit='m/s'
)

# Transform from TETE to TEME frame
vel_teme = vel.transform_to(TEME(obstime=datetime_utc))

# Create DataFrame from transformed coordinates
ephemeris_gps = pd.DataFrame({
    'Datetime_UTC': pd.to_datetime(gps_data["Datetime_UTC"]),
    'pos_x': pos_teme.cartesian.x.value,
    'pos_y': pos_teme.cartesian.y.value,
    'pos_z': pos_teme.cartesian.z.value,
    'vel_x': vel_teme.cartesian.x.value,
    'vel_y': vel_teme.cartesian.y.value,
    'vel_z': vel_teme.cartesian.z.value,
})

# Scale velocity from m/s to km/s
ephemeris_gps['vel_x'] = ephemeris_gps['vel_x'] / 1000.0
ephemeris_gps['vel_y'] = ephemeris_gps['vel_y'] / 1000.0
ephemeris_gps['vel_z'] = ephemeris_gps['vel_z'] / 1000.0

display(ephemeris_gps)

,Datetime_UTC,pos_x,pos_y,pos_z,vel_x,vel_y,vel_z
0,2024-06-06 17:50:01.200,-1734.505155,1054.724715,6534.02166,-3.787005,6.311760,-2.021787
1,2024-06-06 17:50:31.200,-1847.128527,1243.455799,6469.72896,-3.720479,6.269039,-2.264072
2,2024-06-06 17:51:01.200,-1957.693711,1430.801521,6398.20526,-3.649777,6.219425,-2.503648
3,2024-06-06 17:51:31.200,-2066.068105,1616.537405,6319.53764,-3.575008,6.162761,-2.740609
4,2024-06-06 17:52:01.200,-2172.156706,1800.501083,6233.79323,-3.496291,6.099252,-2.974643
...,...,...,...,...,...,...,...
2618,2024-06-07 16:59:02.200,3530.319400,-5627.654713,1696.90092,-1.888192,1.040238,7.315217
2619,2024-06-07 16:59:32.200,3471.718854,-5593.319080,1915.36965,-2.018075,1.248376,7.247894
2620,2024-06-07 17:00:02.200,3409.250724,-5552.755919,2131.70063,-2.146017,1.455550,7.172545
2621,2024-06-07 17:00:32.200,3342.985911,-5506.007496,2345.64762,-2.271625,1.661316,7.089164


In [4]:
# Convert 'Datetime_UTC' column to a NumPy array of Python datetime objects
t = np.array([_t.to_pydatetime() for _t in pd.to_datetime(ephemeris_gps["Datetime_UTC"])])

# Create ephemeris list (needs to be in km and km/s)
ephemeris = [((row["pos_x"], row["pos_y"], row["pos_z"]), (row["vel_x"], row["vel_y"], row["vel_z"])) for idx, row in ephemeris_gps.iterrows()]

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    t = np.array(t) 

In [5]:
# Run the fitter
last_obs = 4320
obs_stride = 1
epoch_obs = 0
lamda = 1e-3 * 0 + 1  # Interesting. The smaller number works, but diverges. This is better
rms_epsilon = 0.002
iterations, solve_sat, elements_coe, sigma, sigmas, dxs, bs, lamdas, b_epoch, b_new_epoch, b, P, A = \
test_tle_fit_normalized_equinoctial(
    t, ephemeris, central_diff=True, last_obs=last_obs, obs_stride=obs_stride, epoch_obs=epoch_obs, lamda=lamda, rms_epsilon=rms_epsilon, debug=True
)

# Optionally thin the observations
tt = t[::obs_stride]
tephemeris = ephemeris[::obs_stride]

if last_obs:
    tt = tt[:last_obs]
    tephemeris = tephemeris[:last_obs]

print(f'Epoch: {tt[epoch_obs]}\n')
tle = '\n'.join(exporter.export_tle(solve_sat.model))
print(tle)

Initial semi-major axis (a) = 6843.314 km
COE elements (original) = [6843.314223712818, 0.00034302962464342044, 1.7043311956795457, 5.289708983062856, 0.8120207389804214, 1.0290124180716649, 1e-06]
Residuals at epoch time [-5.07214112e-01  1.53656017e+00 -7.68621805e+00 -9.44927602e-04
  1.18934801e-03 -3.74628755e-04]
Residual magnitudes at epoch time 7.85469, 0.00156454


#################### ITERATION 1 ####################

Condition number (A): 34467.557
Condition number (ATWA_acc): 298974.1237775711
Lambda:  0.1
Residuals after/before 4.4e+08 < 1e+13
Covariance a: 0.019 m
dx  [ 1.13570656e+01 -4.44289588e-05  4.52703193e-04  1.00217204e-03
  6.18465684e-05 -1.16028453e-04  4.92337094e-03]
COE elements = [6854.671289348646, 0.0004884209618688926, 1.7042313732334509, 5.28965346451578, 1.921041386648816, 6.204234768168642, 0.00492437094104478]
EQN elements = (6854.671289348646, 0.0002929688253488283, 0.0003907995693020426, 0.8485590049740654, -0.9579557013921781, 0.6238836993701632,